In [138]:
import warnings
import itertools
import pickle
import copy
import math
import bisect
import torch
import json
import numpy as np
import pandas as pd
import torch.nn as nn
from transformers import BertModel
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


feature_partition = {
    0: [0,1],
    1: [2,3],
    2: [4,5,6,7,8,9,10,11],
    3: [12,13],
}

In [204]:
def generate_partition(f_part):  #input a dict
    setnum = len(f_part)  #子集數
    num_set = [i for i in range(setnum)]  #子集數list
    cardlist = [len(i) for i in f_part.values()]  #各子集cardinality
    f_list = [i for i in f_part.values()]  #list of list
    all_subsets = []
    for r in range(setnum+1):
        for s in combinations(num_set, r):
            tmp = []
            for t in s:
                tmp.extend(f_list[t])
            all_subsets.append(np.array(sorted(tmp)))
    # Create a hash table to store the index of each subset
    subset_index_table = {}
    for index, value in enumerate(all_subsets):
        subset_index_table[tuple(value)] = index
    return all_subsets, subset_index_table, len(all_subsets)


In [220]:
class Hypercube_wpart:
    '''
    A class to create a hypercube object which stores values of vertices
    '''    
    #輸入維度
    def __init__(self, partition):   #input a dict
        self.f_part = partition
        self.n_dim = len(self.f_part)
        self.elem_count = sum([len(i) for i in self.f_part.values()])
        # vertex_values is a dictionary to store the value of each vertex.
        # Because np.array is not hashable, we use tuple to represent the vertex.
        self.vertex_values = {}
        self.vertices, self.vertex_index, self.vertex_num = generate_partition(self.f_part)  #vertex 上的value一次考慮整個subset
        self.edges, self.edge_num = self.build_edges()
        self.differential_matrix = None
        self.weight_matrix = None
        self.generate_min_l2_norm_matrix()
    
    def build_edges(self):
        num_set = [i for i in range(self.n_dim)]  #子集數list
        s_set = set(num_set)  #轉集合
        cardlist = [len(i) for i in self.f_part.values()]  #各子集cardinality
        f_list = [i for i in self.f_part.values()]  #list of list
        #print(f'Receive {f_list}')
        s_subset = set(tuple(i) for i in f_list)
        edges = []
        for r in range(self.n_dim): 
            for v in combinations(num_set, r):
                v_set = set(v)
                adjunct_v = s_set - v_set
                for new_elem in adjunct_v:
                    d_set = v_set | {new_elem}
                    outlist, inlist = [],[]                    
                    for k in v_set:
                        outlist.extend(f_list[k])
                    for l in d_set:
                        inlist.extend(f_list[l])
                    edges.append(((np.array(sorted(outlist))),np.array(sorted(inlist))))
        return edges, len(edges)
    
    def get_elements(self, index):
        return tuple(self.f_part[index])

    def set_vertex_values(self, vertex_values):         #設置點值
        for v in vertex_values:                         #用鍵值來做查找
            self.vertex_values[v] = vertex_values[v]
        
    def does_edge_exist(self, v1, v2):
        if abs(len(v1)-len(v2))==1:
            interset = np.intersect1d(v1,v2)
            smaller = v1 if len(v1)<len(v2) else v2
            return True if np.array_equal(smaller, interset) else False
        else:
            return False
    
    # Establish the matrix A in the above formula: AX-Y
    def generate_differential_matrix(self):
        if self.differential_matrix is None:
            self.differential_matrix = np.zeros((self.edge_num+1, self.vertex_num))
            for i,v_pair in enumerate(self.edges):
                j = self.vertex_index[tuple(v_pair[1])]
                k = self.vertex_index[tuple(v_pair[0])]
                self.differential_matrix[i][j] = 1
                self.differential_matrix[i][k] = -1
            # Add one more equestion that x_0 = 0 into the matrix form
            self.differential_matrix[-1][0]=1
        return self.differential_matrix

    # Pre-calcuate "W=(A^T*A)^-1*A^T" for the formula "X = ((A^T*A)^-1*A^T)*Y
    def generate_min_l2_norm_matrix(self):
        matrix_A = self.generate_differential_matrix()
        matrix_A_T = np.transpose(matrix_A)
        self.weight_matrix = np.linalg.inv(matrix_A_T @ matrix_A) @ matrix_A_T

    def get_gradient_vector(self):
        gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return gradient_vector      
        
    def get_partial_gradient_vector(self,subset_i):  #feature->subset
        feature_i = self.get_elements(subset_i)
        partial_gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            if (not set(feature_i).issubset(set(v_pair[0]))) and (set(feature_i).issubset(set(v_pair[1]))):
                partial_gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return partial_gradient_vector
    
    def resolve_vi(self, subset_i, phi_0=0):  #feature->subset
        pd = self.get_partial_gradient_vector(subset_i)
        # Append equation x_0=0 at the end of partial gradient vector.
        pd = np.append(pd, phi_0)
        vi = self.weight_matrix @ pd
        # Reconstruct the vertex values
        new_vertices = {}
        for i,v in enumerate(self.vertices):
            new_vertices[tuple(v)] = vi[i]
        return vi, new_vertices

In [194]:
ab = Hypercube_wpart(feature_partition)
print(ab.n_dim)
print(ab.elem_count)
print(ab.vertices)
print(ab.edges)

4
14
[array([], dtype=float64), array([0, 1]), array([2, 3]), array([ 4,  5,  6,  7,  8,  9, 10, 11]), array([12, 13]), array([0, 1, 2, 3]), array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11]), array([ 0,  1, 12, 13]), array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([ 2,  3, 12, 13]), array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([ 0,  1,  2,  3, 12, 13]), array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])]
[(array([], dtype=float64), array([0, 1])), (array([], dtype=float64), array([2, 3])), (array([], dtype=float64), array([ 4,  5,  6,  7,  8,  9, 10, 11])), (array([], dtype=float64), array([12, 13])), (array([0, 1]), array([0, 1, 2, 3])), (array([0, 1]), array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11])), (array([0, 1]), array([ 0,  1, 12, 13])), (array([2, 3]), array([0, 1, 2, 3])), (array([

In [180]:
testdict = {
    0: [0,1],
    1: [2],
    2: [3,4,5],
}

subset_i = 1

hyper = Hypercube_wpart(testdict)
vertices = {}
subsets, idx, _ = generate_partition(testdict)

values = [0,1,0,0,1,1,2,3]
for v in subsets:
    vertices[tuple(v)] = values.pop(0)
hyper.set_vertex_values(vertices)

pd = hyper.get_partial_gradient_vector(subset_i)
print(f'partial gradient of subset {subset_i}:', pd)
vi, new_vs = hyper.resolve_vi(subset_i)
print(f'argmin vi:',vi)
print(new_vs)

h1 = Hypercube_wpart(testdict)
h1.set_vertex_values(new_vs)
dvi = h1.get_gradient_vector()
print('gradient of vi:',dvi)
print('residual:',dvi-pd)

partial gradient of subset 1: [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0.]
argmin vi: [ 4.4408921e-16  8.8817842e-16  5.0000000e-01 -5.0000000e-01
  5.0000000e-01 -5.0000000e-01  1.0000000e+00  1.0000000e+00]
{(): 4.440892098500626e-16, (0, 1): 8.881784197001252e-16, (2,): 0.5000000000000004, (3, 4, 5): -0.5, (0, 1, 2): 0.5000000000000004, (0, 1, 3, 4, 5): -0.4999999999999991, (2, 3, 4, 5): 1.0000000000000009, (0, 1, 2, 3, 4, 5): 1.0000000000000004}
gradient of vi: [ 4.4408921e-16  5.0000000e-01 -5.0000000e-01  5.0000000e-01
 -5.0000000e-01  0.0000000e+00  5.0000000e-01  8.8817842e-16
  1.5000000e+00  5.0000000e-01  1.5000000e+00 -4.4408921e-16]
residual: [ 4.4408921e-16  5.0000000e-01 -5.0000000e-01  5.0000000e-01
 -5.0000000e-01  0.0000000e+00  5.0000000e-01  8.8817842e-16
 -5.0000000e-01  5.0000000e-01 -5.0000000e-01 -4.4408921e-16]


In [224]:
testdict = {
    0: [0,3],
    1: [1,4],
    2: [2],
}

subset_i = 1
subsets, idx, _ = generate_partition(testdict)
print(subsets)
print(idx)
hyper = Hypercube_wpart(testdict)
vertices = {}


values = [0,1,0,0,1,1,2,3]
for v in subsets:
    vertices[tuple(v)] = values.pop(0)
hyper.set_vertex_values(vertices)

pd = hyper.get_partial_gradient_vector(subset_i)
print(f'partial gradient of subset {subset_i}:', pd)
vi, new_vs = hyper.resolve_vi(subset_i)
print(f'argmin vi:',vi)
print(new_vs)

h1 = Hypercube_wpart(testdict)
h1.set_vertex_values(new_vs)
dvi = h1.get_gradient_vector()
print('gradient of vi:',dvi)
print('residual:',dvi-pd)

[array([], dtype=float64), array([0, 3]), array([1, 4]), array([2]), array([0, 1, 3, 4]), array([0, 2, 3]), array([1, 2, 4]), array([0, 1, 2, 3, 4])]
{(): 0, (0, 3): 1, (1, 4): 2, (2,): 3, (0, 1, 3, 4): 4, (0, 2, 3): 5, (1, 2, 4): 6, (0, 1, 2, 3, 4): 7}
partial gradient of subset 1: [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0.]
argmin vi: [ 4.4408921e-16  8.8817842e-16  5.0000000e-01 -5.0000000e-01
  5.0000000e-01 -5.0000000e-01  1.0000000e+00  1.0000000e+00]
{(): 4.440892098500626e-16, (0, 3): 8.881784197001252e-16, (1, 4): 0.5000000000000004, (2,): -0.5, (0, 1, 3, 4): 0.5000000000000004, (0, 2, 3): -0.4999999999999991, (1, 2, 4): 1.0000000000000009, (0, 1, 2, 3, 4): 1.0000000000000004}
gradient of vi: [ 4.4408921e-16  5.0000000e-01 -5.0000000e-01  5.0000000e-01
 -5.0000000e-01  0.0000000e+00  5.0000000e-01  8.8817842e-16
  1.5000000e+00  5.0000000e-01  1.5000000e+00 -4.4408921e-16]
residual: [ 4.4408921e-16  5.0000000e-01 -5.0000000e-01  5.0000000e-01
 -5.0000000e-01  0.0000000e+00  5.000000